# Price Agent

The ``PriceThresholdSwapAgent`` is a reactive DeFi Agent that continuously monitors token prices in a specified liquidity pool, such as a Uniswap V2 pair for USDC/WETH. It automatically triggers a predefined swap (e.g., selling WETH for USDC) when the price exceeds a user-set threshold, integrating real-time event feeds for efficient execution

* To download notebook to this tutorial, see [here](https://github.com/defipy-devs/defipy-docs/blob/main/docs/onchain/onchain/price_threshold_swap.ipynb) 

In [13]:
from defipy import *
from web3scout import *

### 📘 Notable Classes

---

* **Class**: 📘 ``defipy.agents.config.PriceThresholdConfig`` 
    * **Purpose**: Configuration for price threshold swap agent.
        * **Parameters**:
            * `threshold`: Price threshold to check (`float`).
            * `pool_address`: UniV2 pool address (`str`).
            * `provider_url`: Provider URL (eg, infura.io)(`str`).
            * `platform`: Platform where pool resides (eg, uniswap) (`str`).
            * `abi_name`: ABI handle pointing to pool contract in JSON (eg, UniswapV2Pair) (`str`).

---
         
* **Class**: 📘 ``defipy.agents.PriceThresholdSwapAgent`` 
    * **Purpose**: Reactive DeFi agent for determining price threshold.
        * **Parameters**:
            * `config`: Agent configuration parameters (`PriceThresholdConfig`).
    * **Methods**:
        * ``apply()``
            * **Purpose**: Apply price threshold agent   
        * ``get_current_price(tkn1_over_tkn0: bool)``
            * **Purpose**: Get current price of pool  
            * **Parameters**:
                * `tkn1_over_tkn0`: price = tkn1/tkn0 (true) or price = tkn0/tkn1 (false)(`bool`).
        * ``check_condition(threshold: float, tkn1_over_tkn0: bool)``
            * **Purpose**: Calculate initial token value of position   
            * **Parameters**:
                * `threshold`: override config price threshold (optional) (`float`).
                * `tkn1_over_tkn0`: price = tkn1/tkn0 (true) or price = tkn0/tkn1 (false) (optional) (`float`).
        * ``get_w3()``
            * **Purpose**: Getter function for web3.py connector object
        * ``get_abi()``
            * **Purpose**: Getter function for ABI json data
        * ``get_contract_instance()``
            * **Purpose**: Getter function for pool contract instance
        * ``get_lp_data()``
            * **Purpose**: Getter function for pool data associated with `pool_address` from config


### Setup Agent

In [17]:
price_threshold = 3000.0
pair_address = "0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc"
provider_url = "https://mainnet.infura.io/v3/9624e3e5c40f4ac3958b79fa5aa2562d"
platform = Platform.AGNOSTIC
abi_name = JSONContract.UniswapV2Pair

config = PriceThresholdConfig(
    threshold = price_threshold,
    pool_address = pair_address,
    provider_url = provider_url,
    platform = platform,
    abi_name = abi_name,
)

agent = PriceThresholdSwapAgent(config)

print(f"Monitoring price movements @ pool address {pair_address}")

Monitoring price movements @ pool address 0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc


### Agent Run #1

In [15]:
# Apply agent
agent.apply()

# Grab agent data
tkn1_over_tkn0 = True
price = agent.get_current_price(True)
price_condition_pass = agent.check_condition()
contract_instance = agent.get_contract_instance()
lp_data = agent.get_lp_data()

tkn0 = lp_data.tkn0; tkn1 = lp_data.tkn1; reserves = lp_data.reserves

# Print agent data
print("---------------------------------------------------------------------------------------")
print(f"Agent data @ pool address {pair_address}")
print("---------------------------------------------------------------------------------------")
print(f"reserve0 = {reserves[0]/(10**tkn0.token_decimal):.2f} {tkn0.token_name} @ token address {tkn0.token_addr}")
print(f"reserve1 = {reserves[1]/(10**tkn1.token_decimal):.2f} {tkn1.token_name} @ token address {tkn1.token_addr}")

if(tkn1_over_tkn0):
    price = (reserves[0] / reserves[1]) * (10 ** (tkn1.token_decimal - tkn0.token_decimal))
    print(f"\n{tkn1.token_name} Price in {tkn0.token_name}: {price}")
    print(f"Threshold PASS, {tkn1.token_name} Price > {price_threshold}: {price_condition_pass}")
else:
    price = (reserves[1] / reserves[0]) * (10 ** (tkn0.token_decimal - tkn1.token_decimal))
    print(f"\n{tkn0.token_name} Price in {tkn1.token_name}: {price}")
    print(f"Threshold PASS, {tkn0.token_name} Price > {price_threshold}: {price_condition_pass}")

---------------------------------------------------------------------------------------
Agent data @ pool address 0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc
---------------------------------------------------------------------------------------
reserve0 = 14722892.12 USDC @ token address 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
reserve1 = 3913.53 WETH @ token address 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2

WETH Price in USDC: 3762.0484533085046
Threshold PASS, WETH Price > 3000.0: True


### Agent Run #2

In [16]:
price_threshold = 0.0003
tkn1_over_tkn0 = False

# Apply agent
agent.apply()

# Grab agent data
price = agent.get_current_price(True)
price_condition_pass = agent.check_condition(price_threshold, tkn1_over_tkn0)
contract_instance = agent.get_contract_instance()
lp_data = agent.get_lp_data()

tkn0 = lp_data.tkn0; tkn1 = lp_data.tkn1; reserves = lp_data.reserves

# Print agent data
print("---------------------------------------------------------------------------------------")
print(f"Agent data @ pool address {pair_address}")
print("---------------------------------------------------------------------------------------")
print(f"reserve0 = {reserves[0]/(10**tkn0.token_decimal):.2f} {tkn0.token_name} @ token address {tkn0.token_addr}")
print(f"reserve1 = {reserves[1]/(10**tkn1.token_decimal):.2f} {tkn1.token_name} @ token address {tkn1.token_addr}")

if(tkn1_over_tkn0):
    price = (reserves[0] / reserves[1]) * (10 ** (tkn1.token_decimal - tkn0.token_decimal))
    print(f"\n{tkn1.token_name} Price in {tkn0.token_name}: {price}")
    print(f"Threshold PASS, {tkn1.token_name} Price > {price_threshold}: {price_condition_pass}")
else:
    price = (reserves[1] / reserves[0]) * (10 ** (tkn0.token_decimal - tkn1.token_decimal))
    print(f"\n{tkn0.token_name} Price in {tkn1.token_name}: {price}")
    print(f"Threshold PASS, {tkn0.token_name} Price > {price_threshold}: {price_condition_pass}")

---------------------------------------------------------------------------------------
Agent data @ pool address 0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc
---------------------------------------------------------------------------------------
reserve0 = 14722892.12 USDC @ token address 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
reserve1 = 3913.53 WETH @ token address 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2

USDC Price in WETH: 0.0002658126317114703
Threshold PASS, USDC Price > 0.0003: False
